# 第2课：装饰器

## 学习目标
- 理解装饰器的概念
- 掌握函数装饰器的编写
- 了解带参数的装饰器
- 学会类装饰器

## 1. 装饰器基础

装饰器是一个接收函数并返回新函数的函数，用于在不修改原函数代码的情况下增加功能。

In [ ]:
# 函数是一等公民
def greet(name):
    return f"Hello, {name}!"

# 函数可以赋值给变量
say_hello = greet
print(say_hello("World"))

# 函数可以作为参数
def call_func(func, arg):
    return func(arg)

print(call_func(greet, "Python"))

In [ ]:
# 简单装饰器
def simple_decorator(func):
    def wrapper():
        print("函数执行前")
        func()
        print("函数执行后")
    return wrapper

@simple_decorator
def say_hello():
    print("Hello!")

say_hello()

In [ ]:
# @ 语法等价于
def say_hello():
    print("Hello!")

say_hello = simple_decorator(say_hello)
say_hello()

## 2. 带参数的函数装饰器

In [ ]:
# 处理任意参数
def logger(func):
    def wrapper(*args, **kwargs):
        print(f"调用函数: {func.__name__}")
        print(f"参数: args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)
        print(f"返回值: {result}")
        return result
    return wrapper

@logger
def add(a, b):
    return a + b

@logger
def greet(name, greeting="Hello"):
    return f"{greeting}, {name}!"

add(3, 5)
print()
greet("Python", greeting="Hi")

In [ ]:
# 保留原函数信息
from functools import wraps

def logger(func):
    @wraps(func)  # 保留原函数的元信息
    def wrapper(*args, **kwargs):
        """wrapper 文档"""
        print(f"调用: {func.__name__}")
        return func(*args, **kwargs)
    return wrapper

@logger
def add(a, b):
    """两数相加"""
    return a + b

print(f"函数名: {add.__name__}")
print(f"文档: {add.__doc__}")

## 3. 带参数的装饰器

In [ ]:
# 装饰器工厂
def repeat(times):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for _ in range(times):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator

@repeat(3)
def say_hello(name):
    print(f"Hello, {name}!")

say_hello("World")

In [ ]:
# 计时装饰器
import time

def timer(unit="s"):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            start = time.time()
            result = func(*args, **kwargs)
            end = time.time()
            
            elapsed = end - start
            if unit == "ms":
                elapsed *= 1000
            print(f"{func.__name__} 执行时间: {elapsed:.4f} {unit}")
            return result
        return wrapper
    return decorator

@timer(unit="ms")
def slow_function():
    time.sleep(0.1)
    return "完成"

slow_function()

## 4. 常用装饰器示例

In [ ]:
# 缓存装饰器
def cache(func):
    memo = {}
    @wraps(func)
    def wrapper(*args):
        if args not in memo:
            memo[args] = func(*args)
        return memo[args]
    return wrapper

@cache
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

print(f"fibonacci(35) = {fibonacci(35)}")

In [ ]:
# 使用内置的 lru_cache
from functools import lru_cache

@lru_cache(maxsize=128)
def fibonacci2(n):
    if n < 2:
        return n
    return fibonacci2(n-1) + fibonacci2(n-2)

print(f"fibonacci2(50) = {fibonacci2(50)}")
print(f"缓存信息: {fibonacci2.cache_info()}")

In [ ]:
# 重试装饰器
import random

def retry(max_attempts=3, exceptions=(Exception,)):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for attempt in range(max_attempts):
                try:
                    return func(*args, **kwargs)
                except exceptions as e:
                    print(f"尝试 {attempt + 1} 失败: {e}")
                    if attempt == max_attempts - 1:
                        raise
        return wrapper
    return decorator

@retry(max_attempts=5)
def unreliable_function():
    if random.random() < 0.7:
        raise ValueError("随机失败")
    return "成功!"

try:
    result = unreliable_function()
    print(result)
except ValueError:
    print("最终失败")

In [ ]:
# 类型检查装饰器
def type_check(**expected_types):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # 检查位置参数
            import inspect
            sig = inspect.signature(func)
            params = list(sig.parameters.keys())
            
            for i, arg in enumerate(args):
                param_name = params[i]
                if param_name in expected_types:
                    if not isinstance(arg, expected_types[param_name]):
                        raise TypeError(f"{param_name} 应为 {expected_types[param_name]}")
            
            # 检查关键字参数
            for key, value in kwargs.items():
                if key in expected_types:
                    if not isinstance(value, expected_types[key]):
                        raise TypeError(f"{key} 应为 {expected_types[key]}")
            
            return func(*args, **kwargs)
        return wrapper
    return decorator

@type_check(a=int, b=int)
def add(a, b):
    return a + b

print(add(1, 2))
# print(add("1", 2))  # 会抛出 TypeError

## 5. 类装饰器

In [ ]:
# 使用类实现装饰器
class CountCalls:
    def __init__(self, func):
        self.func = func
        self.count = 0
    
    def __call__(self, *args, **kwargs):
        self.count += 1
        print(f"{self.func.__name__} 被调用了 {self.count} 次")
        return self.func(*args, **kwargs)

@CountCalls
def say_hello():
    print("Hello!")

say_hello()
say_hello()
say_hello()

In [ ]:
# 装饰类的装饰器
def singleton(cls):
    """单例模式装饰器"""
    instances = {}
    @wraps(cls)
    def wrapper(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    return wrapper

@singleton
class Database:
    def __init__(self):
        print("初始化数据库连接")
        self.connected = True

db1 = Database()
db2 = Database()
print(f"db1 is db2: {db1 is db2}")

## 6. 多个装饰器叠加

In [ ]:
def bold(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        return f"<b>{func(*args, **kwargs)}</b>"
    return wrapper

def italic(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        return f"<i>{func(*args, **kwargs)}</i>"
    return wrapper

@bold
@italic
def greet(name):
    return f"Hello, {name}"

# 执行顺序：greet -> italic -> bold
print(greet("World"))

## 7. 练习题

### 练习 1：权限检查装饰器
创建一个检查用户权限的装饰器

In [ ]:
def require_permission(permission):
    # 在这里编写代码
    pass

# 测试
current_user_permissions = ["read", "write"]

@require_permission("admin")
def delete_user(user_id):
    return f"删除用户 {user_id}"

### 练习 2：日志装饰器
创建一个可配置的日志装饰器

In [ ]:
def log(level="INFO"):
    # 在这里编写代码
    pass

## 8. 本课小结

1. **装饰器**：接收函数返回新函数
2. **@语法**：语法糖，等价于 `func = decorator(func)`
3. **functools.wraps**：保留原函数元信息
4. **带参数装饰器**：装饰器工厂模式
5. **类装饰器**：使用 `__call__` 方法
6. **装饰器叠加**：从下往上应用，从上往下执行